In [ ]:
!pip install --upgrade pip
!pip install mysql-connector-python

In [ ]:
import mysql.connector as connector

connection = connector.connect(
    user="dragonalpha",
    password="killthemall",
    database="LittleLemonDB"
)

In [ ]:
cursor = connection.cursor()

connection.database

In [ ]:
cursor.execute("""CREATE VIEW OrdersView AS
SELECT OrderID, Quantity, TotalCost
FROM orders
WHERE Quantity > 2""")

In [ ]:
show_tables_query = """SHOW tables"""
cursor.execute(show_tables_query)
results = cursor.fetchall()
cols = cursor.column_names[1:]
print("Tables in the LittleLemonDB database:")
print("--------------------------------------")
print(cols)
for result in results:
    print(result)

In [ ]:
orders_query = """SELECT * FROM customerdetails""" 
cursor.execute(orders_query)
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

In [ ]:
orders_query = """SELECT customers.CustomerID, customers.Names, orders.OrderID, orders.TotalCost, menu.Starters, menu.Cuisines, menu.Courses, menu.Drinks, menu.Desserts
FROM customerdetails customers INNER JOIN orders
ON customers.CustomerID = orders.customerID
INNER JOIN menu ON orders.OrderID = menu.OrderID 
WHERE orders.TotalCost > 150  ORDER BY orders.TotalCost""" 
cursor.execute(orders_query)
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

In [ ]:
orders_query = """
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(quantity) AS 'Max Quantity in Order' FROM orders;
END
"""

cursor.execute(orders_query)


In [ ]:
orders_query = """PREPARE GetOrderDetail FROM 
'SELECT OrderID, Quantity, TotalCost FROM Orders WHERE CustomerID = ?'"""
cursor.execute(orders_query)

In [ ]:
orders_query = """
CREATE PROCEDURE CancelOrder(IN orderId INT)
BEGIN
    DELETE FROM Orders WHERE OrderID = orderId;
END
"""

cursor.execute(orders_query)


In [ ]:
orders_query = """
CREATE PROCEDURE CheckBooking(IN booking_date DATE, IN table_number INT)
BEGIN
    DECLARE bookedTable INT DEFAULT 0;
    
    SELECT COUNT(*)
    INTO bookedTable
    FROM Bookings
    WHERE `Date` = booking_date AND TableNumber = table_number;
    
    IF bookedTable > 0 THEN
        SELECT CONCAT('Table ', table_number, ' is already booked') AS 'Booking status';
    ELSE 
        SELECT CONCAT('Table ', table_number, ' is not booked') AS 'Booking status';
    END IF;
END;
"""

cursor.execute(orders_query)

In [ ]:
orders_query = """
CREATE PROCEDURE AddValidBooking(IN bookingDate DATE, IN tableNumber INT, IN customerName VARCHAR(255))
BEGIN
    DECLARE existingBookingCount INT;

    -- Check if the table is already booked
    SELECT COUNT(*) INTO existingBookingCount
    FROM Bookings
    WHERE `Date` = bookingDate AND TableNumber = tableNumber;

    -- Start a transaction
    START TRANSACTION;

    IF existingBookingCount > 0 THEN
        -- The table is already booked, so rollback the transaction
        ROLLBACK;
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Table is already booked for this date.';
    ELSE
        -- The table is available, so insert the new booking
        INSERT INTO Bookings (`Date`, TableNumber, CustomerName)
        VALUES (bookingDate, tableNumber, customerName);

        -- Commit the transaction
        COMMIT;
    END IF;
END
"""

cursor.execute(orders_query)

In [ ]:
orders_query = """
CREATE PROCEDURE UpdateBooking(IN booking_date DATE, IN booking_id INT)
BEGIN
  UPDATE bookings
  SET `Date` = booking_date
  WHERE BookingID = booking_id;

  SELECT CONCAT('Booking ', booking_id, ' updated') AS Confirmation;
END
"""

cursor.execute(orders_query)

In [ ]:
orders_query = """
CREATE PROCEDURE CancelBooking(IN booking_id_to_cancel INT)
BEGIN
  DELETE FROM Bookings
  WHERE BookingID = booking_id_to_cancel;

  SELECT CONCAT('Booking ', booking_id_to_cancel, ' cancelled') AS Confirmation;
END
"""

cursor.execute(orders_query)